<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'TFrame', 'LatestQtr', 'StarStock', 'BizFinTech', 'Conviction', 'Category', 'InFolio', 'XIRR', 'Remarks']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2024-12-19 11:02:26
-------------------
qualified stocks: 87
with latest results: 83
still star stocks: 42
-------------------
Initial Investment:  1.01 C
CY Investment:  1.14 C
Reserve:  2.50 K
Current:  1.19 C
-------------------
Today PnL: -1.15 L (-0.96%)
Current PnL: -5.73 L (-5.02%)
CY Booked + Current PnL: 4.54 L (3.98%)
-------------------
Total profit:  6.92 L
Total loss:  -12.65 L
-------------------
Total Booked + Current PnL: 17.65 L (17.47%)
Total Booked PnL: 23.38 L (23.14%)
Curr Year Booked PnL: 10.27 L (8.64%)
Prev Year Booked PnL: 13.11 L (12.98%)
Est FTT:  1.94 C
Est FTT PnL: 75.48 L (63.46%)
-------------------
Est LTT:  2.46 C
Est LTT PnL: 1.27 C (106.58%)
Deployed:  1.01 C
Current:  1.19 C
CAGR/XIRR %: 14.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'Close', 'FTT', 'Dev%_PE', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol   AvgCost     Close       FTT  Dev%_PE    RSP   Current  \
53     PAGEIND  38829.08  49339.00  52350.84     8.86  89.66  197356.0   
46  LALPATHLAB   2519.90   3021.80   3689.00   -12.41  35.63  108785.0   
77         TCS   4164.02   4327.70   4998.00     6.55  78.16  168780.0   
59    RELIANCE   1322.24   1238.05   1590.07    -6.51  36.78   94092.0   
48        LTIM   5664.39   6292.45   7262.39    19.30  75.86  188774.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
53      42040.0  12039.0        0.47         27.07   6.10  34.82  11.0   
46      18069.0  24020.0       -0.56         19.92  22.08  46.39  26.0   
77       6383.0  26144.0       -0.46          3.93  15.49  20.03  15.0   
59      -6398.0  26750.0       -1.21         -6.37  28.43  20.26  95.0   
48      18842.0  29090.0       -4.28         11.09  15.41  28.21  12.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
53     3.49        1.66    50.86      X40      ATH    APPARELS  
46     0.75        0.91    51.15     X40N      NTT  HEALTHCARE  
77     0.24        1.42    20.00      X40      BTT          IT  
59    -0.24        0.79     1.23      X40      ATH  REFINERIES  
48     0.65        1.59    38.67     X200      ATH          IT

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    Close      FTT  Dev%_PE    RSP   Current  \
15  BAYERCROP  4346.57  5784.00   8325.0    24.55  58.62   92544.0   
49     LUXIND  1448.19  2063.65   2880.0    55.52  97.70  196047.0   
78  TEAMLEASE  2557.47  2922.60   4990.0    -8.90  77.01  122749.0   
40      JCHAC  2282.24  1826.00   3542.0    47.07  18.97  102256.0   
19       CERA  9059.90  7545.10  12390.0    -2.61  68.97   75451.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
15      22999.0  40655.0        1.17         33.07  43.93  91.53  20.0   
49      58469.0  77556.0        1.05         42.50  39.56  98.87  16.0   
78      15335.0  86833.0        0.68         14.28  70.74  95.11   5.0   
40     -25549.0  96100.0        0.58        -19.99  93.98  55.20 -69.0   
19     -15148.0  48447.0        0.53        -16.72  64.21  36.76 -67.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
15     0.57        0.78    18.48     X40N      BTT  CHEMICALS  
49     0.75        1.65    92.26     X40N      NTT   TEXTILES  
78     0.18        1.03     9.69     X40N      NTT       MISC  
40    -0.27        0.86    91.38     X40N      BTT         AC  
19    -0.31        0.63    14.55     X40N      BTT   CERAMICS

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost    Close      FTT  Dev%_PE    RSP   Current  \
48        LTIM  5664.39  6292.45  7262.39    19.30  75.86  188774.0   
84  VAIBHAVGBL   350.21   311.40   670.00     8.67  95.40  135148.0   
5   ASIANPAINT  2961.56  2282.90  4250.00   -37.38   6.90  155237.0   
41      JIOFIN   320.24   316.80   387.00    -4.96  34.48  156499.0   
10  BAJFINANCE  7055.29  6911.20  9150.00   -31.70  64.37  214247.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  XIRR  \
48      18842.0   29090.0       -4.28         11.09   15.41  28.21  12.0   
84     -16843.0  155636.0       -2.93        -11.08  115.16  91.31 -27.0   
5      -46149.0  133768.0       -2.67        -22.92   86.17  43.51 -12.0   
41      -1700.0   34680.0       -2.40         -1.07   22.16  20.85  85.0   
10      -4467.0   69395.0       -2.31         -2.04   32.39  29.69   8.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
48     0.65        1.59    38.67     X200      ATH         IT  
84    -0.11        1.14    16.67       XR      NTT  JEWELLERY  
5     -0.34        1.31     0.00      X40      BTT     PAINTS  
41    -0.05        1.32    36.49      XSP      BTT    FINANCE  
10    -0.06        1.80    10.06      X40      BTT    FINANCE

In [12]:
# top 5 to monitor
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    Close      FTT  Dev%_PE    RSP   Current  \
49      LUXIND  1448.19  2063.65  2880.00    55.52  97.70  196047.0   
15   BAYERCROP  4346.57  5784.00  8325.00    24.55  58.62   92544.0   
52  METROPOLIS  1399.27  2136.00  3102.00    23.99  70.11  143112.0   
81  TTKPRESTIG   769.29   844.20  1149.24    23.63  25.29  110590.0   
72       SUNTV   536.47   715.55  1092.00    20.38  18.97  130946.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  XIRR  \
49      58469.0  77556.0        1.05         42.50  39.56   98.87  16.0   
15      22999.0  40655.0        1.17         33.07  43.93   91.53  20.0   
52      49361.0  64715.0        0.21         52.65  45.22  121.69  34.0   
81       9813.0  39956.0       -0.72          9.74  36.13   49.39  16.0   
72      32772.0  68891.0       -0.68         33.38  52.61  103.55  29.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy       Category  
49     0.75        1.65    92.26     X40N      NTT       TEXTILES  
15     0.57        0.78    18.48     X40N      BTT      CHEMICALS  
52     0.76        1.20    42.04       XR      NTT     HEALTHCARE  
81     0.25        0.93    27.03     X40N      ATH       DURABLES  
72     0.48        1.10    26.73     X40N      BTT  ENTERTAINMENT

In [13]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Criteria'] == 'XSP')].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost   Close     FTT  Dev%_PE    RSP   Current  \
39       IRCTC   904.87  804.35  1269.0   -22.44  42.53  125479.0   
83   UNIONBANK   123.87  121.56   163.0   -10.18  65.52  138214.0   
42     JPPOWER    18.73   18.53    26.2   -61.31  96.55  140328.0   
7     ATULAUTO   624.44  593.00   844.0   -10.38  79.31  139948.0   
16  BERGEPAINT   558.54  445.00   680.0   -38.16   9.20  141065.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  XIRR  \
39     -15681.0  72489.0       -1.06        -11.11  57.77  40.24  15.0   
83      -2626.0  47117.0       -1.07         -1.86  34.09  31.59 -35.0   
42      -1514.0  58082.0       -1.59         -1.07  41.39  39.88  79.0   
7       -7420.0  59240.0       -0.59         -5.03  42.33  35.16 -64.0   
16     -35992.0  74496.0       -1.73        -20.33  52.81  21.75 -16.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
39    -0.22        1.06     1.32      XSP      BTT   TRAVEL  
83    -0.06        1.16    12.31      XSP      NTT    BANKS  
42    -0.03        1.18    43.09      XSP      NTT    POWER  
7     -0.13        1.18    24.66      XSP      NTT     AUTO  
16    -0.48        1.19     0.00      XSP      NTT   PAINTS

In [14]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost    Close     FTT  Dev%_PE     RSP   Current  \
67         SFL  1034.30   971.60  1287.0    27.40  100.00  237070.0   
27       GICRE   335.06   444.35   583.0    12.51   98.85  194181.0   
49      LUXIND  1448.19  2063.65  2880.0    55.52   97.70  196047.0   
42     JPPOWER    18.73    18.53    26.2   -61.31   96.55  140328.0   
84  VAIBHAVGBL   350.21   311.40   670.0     8.67   95.40  135148.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%   XIRR  \
67     -15299.0   76953.0        0.12         -6.06   32.46  24.43  -38.0   
27      47760.0   60584.0       -1.19         32.62   31.20  74.00  111.0   
49      58469.0   77556.0        1.05         42.50   39.56  98.87   16.0   
42      -1514.0   58082.0       -1.59         -1.07   41.39  39.88   79.0   
84     -16843.0  155636.0       -2.93        -11.08  115.16  91.31  -27.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
67    -0.20        1.99    24.56      XSP      NTT       MISC  
27     0.79        1.63    50.10      XSP      BTT  INSURANCE  
49     0.75        1.65    92.26     X40N      NTT   TEXTILES  
42    -0.03        1.18    43.09      XSP      NTT      POWER  
84    -0.11        1.14    16.67       XR      NTT  JEWELLERY

In [15]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.84
1     25       42.13
2     50       72.78

In [16]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XSP     39.38
X40     21.90
X40N    15.30
XR      14.92
AR       5.42
X200     1.59
SR       1.49
Name: CurrAlloc%, dtype: float64

In [17]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%     FTT%
Category                                        
BANKS               14.75         -8.83    67.33
FINANCE             12.40        -10.78    65.13
IT                   9.15         -2.21    55.51
MISC                 8.81          3.35    47.35
PAINTS               5.95        -16.55    51.04
CHEMICALS            5.93         -2.67    69.51
HEALTHCARE           4.91         10.85    35.70
INSURANCE            4.36          6.61    43.77
FMCG                 3.99        -10.08    50.54
AUTO                 3.37         -3.00    48.29
FOOTWEAR             3.01        -14.97    90.89
TEXTILES             2.37         18.36    51.58
TRAVEL               2.30        -12.81    58.87
ENTERTAINMENT        2.27          2.84    65.44
CERAMICS             2.13        -12.42    78.62
ENGINEERING          1.73         11.58    22.49
STEEL                1.72         -9.89    91.79
APPARELS             1.66         21.30     6.10
JEWELLERY            1.66        -47.13   126.56
CEMENT               1.30        -21.94    84.67
RESTAURANTS          1.27         27.71    29.64
POWER                1.18         -1.08    41.39
DURABLES             0.93          8.87    36.13
AC                   0.86        -24.99    93.98
PHARMA               0.80         17.06    47.99
REFINERIES           0.79         -6.80    28.43
HOTELS               0.40       -135.80  1333.61

In [18]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
